In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')#忽略一些警告

In [2]:
train=pd.read_csv("data/train_etl.csv")
train.head()

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,房屋朝向,居住状态,卧室数量,厅的数量,...,出租方式,区,位置,地铁线路,地铁站点,距离,装修情况,月租金,log_rent,新朝向
0,1,3072,0.128906,2,0.236364,0.008628,东南,0.0,1,1,...,-1.0,11.0,118.0,2.0,40.0,0.764167,-1.0,5.602716,1.887481,东南
1,1,3152,0.132812,1,0.381818,0.017046,东,0.0,1,0,...,-1.0,10.0,100.0,4.0,58.0,0.709167,-1.0,16.977929,2.889145,东
2,1,5575,0.042969,0,0.290909,0.010593,东南,0.0,2,1,...,-1.0,12.0,130.0,5.0,37.0,0.572500,-1.0,8.998302,2.302415,东南
3,1,3103,0.085938,2,0.581818,0.019199,南,0.0,3,2,...,-1.0,7.0,90.0,2.0,63.0,0.658333,-1.0,5.602716,1.887481,南
4,1,5182,0.214844,0,0.545455,0.010427,东北,0.0,2,1,...,-1.0,3.0,31.0,0.0,0.0,2.000000,-1.0,7.300509,2.116317,东北


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196499 entries, 0 to 196498
Data columns (total 21 columns):
时间          196499 non-null int64
小区名         196499 non-null int64
小区房屋出租数量    196499 non-null float64
楼层          196499 non-null int64
总楼层         196499 non-null float64
房屋面积        196499 non-null float64
房屋朝向        196499 non-null object
居住状态        196499 non-null float64
卧室数量        196499 non-null int64
厅的数量        196499 non-null int64
卫的数量        196499 non-null int64
出租方式        196499 non-null float64
区           196499 non-null float64
位置          196499 non-null float64
地铁线路        196499 non-null float64
地铁站点        196499 non-null float64
距离          196499 non-null float64
装修情况        196499 non-null float64
月租金         196499 non-null float64
log_rent    196499 non-null float64
新朝向         196499 non-null object
dtypes: float64(13), int64(6), object(2)
memory usage: 31.5+ MB


# 根据房间,厅,卫,房屋面积构造新特征

In [4]:
train["房+卫+厅"]=train["卧室数量"]+train["厅的数量"]+train["卫的数量"]
train["房/总"]=train["卧室数量"]/(train["房+卫+厅"]+1)
train["卫/总"]=train["卫的数量"]/(train["房+卫+厅"]+1)
train["厅/总"]=train["厅的数量"]/(train["房+卫+厅"]+1)
train['卧室面积']=train['房屋面积']/(train['卧室数量']+1)#加1是为了防止分母=0出现结果为inf无穷大的现象
train['楼层比']=train['楼层']/(train["总楼层"]+1)#加1是为了防止分母=0出现结果为inf无穷大的现象
train['户型']=train[['卧室数量','厅的数量','卫的数量']].apply(lambda x:str(x['卧室数量'])+str(x['厅的数量'])+str(x['卫的数量']),axis=1)


# 构建租金平均值特征

In [5]:
rent_means=train[['小区名','新朝向','地铁站点','位置','log_rent']].groupby(['小区名','新朝向','地铁站点','位置'],as_index=False).mean()

rent_means.columns=['小区名','新朝向','地铁站点','位置','平均值特征1']
train=pd.merge(train,rent_means,how='left',on=['小区名','新朝向','地铁站点','位置'])

In [6]:
rent_means2=train[['小区名','log_rent']].groupby(['小区名'],as_index=False).mean()

rent_means2.columns=['小区名','小区平均值特征']
train=pd.merge(train,rent_means2,how='left',on=['小区名'])

In [7]:
rent_means3=train[['新朝向','log_rent']].groupby(['新朝向'],as_index=False).mean()

rent_means3.columns=['新朝向','朝向平均值特征']
train=pd.merge(train,rent_means3,how='left',on=['新朝向'])

In [8]:
rent_means4=train[['地铁站点','log_rent']].groupby(['地铁站点'],as_index=False).mean()

rent_means4.columns=['地铁站点','站点平均值特征']
train=pd.merge(train,rent_means4,how='left',on=['地铁站点'])

In [9]:
rent_means5=train[['位置','log_rent']].groupby(['位置'],as_index=False).mean()

rent_means5.columns=['位置','位置平均值特征']
train=pd.merge(train,rent_means5,how='left',on=['位置'])

# 构造是否有地铁

In [10]:
train["有地铁"]=(train["地铁站点"]>-1).map(int)

In [11]:
train.columns

Index(['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', '月租金',
       'log_rent', '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '平均值特征1', '小区平均值特征', '朝向平均值特征', '站点平均值特征', '位置平均值特征', '有地铁'],
      dtype='object')

# 构造聚类特征

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
features=train[['房屋面积','卧室数量','厅的数量','卫的数量','距离','楼层比']]
trans=StandardScaler()
new_features=trans.fit_transform(features)

kmeans=KMeans(n_clusters=5)
kmeans.fit(new_features)
train['聚类特征']=kmeans.predict(new_features).astype(str)

#计算每个聚类特征的月租金平均值
cluster_means=train[['聚类特征','log_rent']].groupby('聚类特征',as_index=False).mean()

cluster_means.columns=['聚类特征','平均值特征2']
train=pd.merge(train,cluster_means,how='left',on=['聚类特征'])

## 保存标准化和聚类模型

In [13]:
import pickle 
with open("data/kmeans.pkl",'wb') as f:
    pickle.dump({
        "std_transer":trans,
        "kmeans":kmeans
    },f)

## 构造地铁线路数特征

In [14]:
lines_count1=train[['小区名','地铁线路']].drop_duplicates().groupby('小区名').count()
lines_count2=train[['位置','地铁线路']].drop_duplicates().groupby('位置').count()
lines_count2.columns=['位置线路数']
lines_count1.columns=['小区线路数']

In [15]:
train=pd.merge(train,lines_count1,how='left',on=['小区名'])
train=pd.merge(train,lines_count2,how='left',on=['位置'])

## 去掉出现数量较少的小区

In [16]:
neighbors=train['小区名'].value_counts()
train['新小区名']=train.apply(lambda x: x['小区名'] if neighbors[x['小区名']]>100 else -1,axis=1)
train['小区条数大于100']=train.apply(lambda x: 1 if neighbors[x['小区名']]>100 else 0,axis=1)

In [17]:
train.info()
train['新小区名'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196499 entries, 0 to 196498
Data columns (total 40 columns):
时间           196499 non-null int64
小区名          196499 non-null int64
小区房屋出租数量     196499 non-null float64
楼层           196499 non-null int64
总楼层          196499 non-null float64
房屋面积         196499 non-null float64
房屋朝向         196499 non-null object
居住状态         196499 non-null float64
卧室数量         196499 non-null int64
厅的数量         196499 non-null int64
卫的数量         196499 non-null int64
出租方式         196499 non-null float64
区            196499 non-null float64
位置           196499 non-null float64
地铁线路         196499 non-null float64
地铁站点         196499 non-null float64
距离           196499 non-null float64
装修情况         196499 non-null float64
月租金          196499 non-null float64
log_rent     196499 non-null float64
新朝向          196499 non-null object
房+卫+厅        196499 non-null int64
房/总          196499 non-null float64
卫/总          196499 non-null float64
厅/总          1964

-1       72488
 5512     1880
 1085     1155
 5208     1136
 6221     1066
         ...  
 244       102
 6461      102
 5906      102
 1654      102
 5196      101
Name: 新小区名, Length: 512, dtype: int64

# 转换类型

In [41]:
#将离散特征转换成字符串类型
colunms = ['时间', '小区名', '居住状态', '出租方式', '区','位置','地铁线路','地铁站点','装修情况']
for col in colunms:
    train[col] = train[col].astype(str)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196499 entries, 0 to 196498
Data columns (total 34 columns):
时间          196499 non-null object
小区名         196499 non-null object
小区房屋出租数量    196499 non-null float64
楼层          196499 non-null int64
总楼层         196499 non-null float64
房屋面积        196499 non-null float64
房屋朝向        196499 non-null object
居住状态        196499 non-null object
卧室数量        196499 non-null int64
厅的数量        196499 non-null int64
卫的数量        196499 non-null int64
出租方式        196499 non-null object
区           196499 non-null object
位置          196499 non-null object
地铁线路        196499 non-null object
地铁站点        196499 non-null object
距离          196499 non-null float64
装修情况        196499 non-null object
月租金         196499 non-null float64
log_rent    196499 non-null float64
新朝向         196499 non-null object
房+卫+厅       196499 non-null int64
房/总         196499 non-null float64
卫/总         196499 non-null float64
厅/总         196499 non-null float64
卧室面积      

In [18]:
#保存处理后的数据
train.to_csv("data/onehot_feature.csv")

In [11]:
x_columns=['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '有地铁']
x=train[x_columns]
y=train['月租金']

# 构建数据清洗和特征工程函数

In [61]:
#     'dist_value_for_fill':dist_value_for_fill,
#     'line_value_for_fill':line_value_for_fill,
#     'station_value_for_fill':station_value_for_fill,
#     'area_value_for_fill':area_value_for_fill,
#     'position_value_for_fill':position_value_for_fill,
#     'state_value_for_fill':state_value_for_fill,
#     'decration_value_for_fill':decration_value_for_fill,
#     'rent_value_for_fill':rent_value_for_fill,
#     'ratio_by_neighbor':ratio_by_neighbor,
#     'ratio_mode':ratio_mode,
#     'time_for_fill':time_for_fill,
#     'neighbor_for_fill':neighbor_for_fill,
#     'ting_for_fill':ting_for_fill,
#     'wei_for_fill':wei_for_fill,
#     'bed_for_fill':bed_for_fill,
#     'direction_for_fill':direction_for_fill,
#     'mianji_for_fill':mianji_for_fill,
#     'louceng_for_fill':louceng_for_fill,
#     'zonglouceng_for_fill':zonglouceng_for_fill
# def process(x,values,models):
    
#     x=x.to_dict()
#     keys=['时间', '小区名', '楼层', '总楼层', '房屋面积',  '居住状态', '卧室数量',
#        '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况']
#     #原特征缺失值填充
#     for key in keys:
#         if pd.isna(x[key]):
#             x[key]=values[key]
#     #小区房屋出租数量
#     if pd.isna(str(x['小区房屋出租数量'])):
#         if x['小区名'] in values['ratio_by_neighbor']:
#             x['小区房屋出租数量']=values['ratio_by_neighbor'][x['小区名']]
#         else:
#             x['小区房屋出租数量']=values['小区房屋出租数量']
#     #房屋朝向问题
# #     print(x['房屋朝向'])
#     if pd.isna(x['房屋朝向']):
#         x['新朝向']=values['新朝向']
#     else:
#         chaoxiang=x['房屋朝向'].split(" ")[0]
#         if chaoxiang in values['所有朝向']:
#             x['新朝向']=chaoxiang
#         else:
#             x['新朝向']=values['新朝向']
    
#     #构造特征
#     x["房+卫+厅"]=x["卧室数量"]+x["厅的数量"]+x["卫的数量"]
#     x["房/总"]=x["卧室数量"]/x["房+卫+厅"]
#     x["卫/总"]=x["卫的数量"]/x["房+卫+厅"]
#     x["厅/总"]=x["厅的数量"]/x["房+卫+厅"]
#     x['卧室面积']=x['房屋面积']/x['卧室数量']
#     x['楼层比']=x['楼层']/(x["总楼层"]+1)#加1是为了防止分母=0出现结果为inf无穷大的现象
#     x['户型']=str(x['卧室数量'])+str(x['厅的数量'])+str(x['卫的数量'])
#     if x["地铁站点"]>-1:
#         x["有地铁"]=1
#     else:
#         x["有地铁"]=0
    
#     #构造聚类特征
#     features=np.array([[x['房屋面积'],x['卧室数量'],x['厅的数量'],x['卫的数量'],x['距离'],x['楼层比']]])
#     models['std_transer'].transform(features)

#     x['聚类特征']=models['kmeans'].predict(new_features).astype(str)[0]
#     return pd.Series(x)


In [66]:
test=pd.read_csv('data/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56279 entries, 0 to 56278
Data columns (total 19 columns):
id          56279 non-null int64
时间          56279 non-null int64
小区名         56279 non-null int64
小区房屋出租数量    56257 non-null float64
楼层          56279 non-null int64
总楼层         56279 non-null float64
房屋面积        56279 non-null float64
房屋朝向        56279 non-null object
居住状态        4483 non-null float64
卧室数量        56279 non-null int64
厅的数量        56279 non-null int64
卫的数量        56279 non-null int64
出租方式        4971 non-null float64
区           56269 non-null float64
位置          56269 non-null float64
地铁线路        26494 non-null float64
地铁站点        26494 non-null float64
距离          26494 non-null float64
装修情况        4207 non-null float64
dtypes: float64(11), int64(7), object(1)
memory usage: 8.2+ MB


In [46]:
with open("data/kmeans.pkl",'rb') as f:
    models=pickle.load(f)

In [47]:
with open("data/values.pkl",'rb') as f:
    values=pickle.load(f)

In [65]:
test.iloc[:100,:].apply(process,models=models,values=values,axis=1)

,id,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,房屋朝向,居住状态,卧室数量,...,新朝向,房+卫+厅,房/总,卫/总,厅/总,卧室面积,楼层比,户型,有地铁,聚类特征
0,1,4,6011,0.382812,1,0.600000,0.007117,东,3.0,2,...,东,4,0.500000,0.250000,0.250000,0.003558,0.625000,211,1,2
1,2,4,1697,0.152344,1,0.472727,0.007448,东,0.0,2,...,东,4,0.500000,0.250000,0.250000,0.003724,0.679012,211,1,2
2,3,4,754,0.207031,2,0.709091,0.014068,东南,0.0,3,...,东南,7,0.428571,0.285714,0.285714,0.004689,1.170213,322,1,2
3,4,4,1285,0.011719,0,0.090909,0.008937,南,0.0,2,...,南,4,0.500000,0.250000,0.250000,0.004469,0.000000,211,1,2
4,5,4,4984,0.035156,1,0.218182,0.008606,东南,0.0,2,...,东南,4,0.500000,0.250000,0.250000,0.004303,0.820896,211,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,4,5239,0.433594,0,0.545455,0.006202,北,0.0,1,...,北,2,0.500000,0.500000,0.000000,0.006202,0.000000,101,1,2
96,97,4,1027,0.046875,0,0.727273,0.007779,南,0.0,2,...,南,4,0.500000,0.250000,0.250000,0.003889,0.000000,211,1,2
97,98,4,300,0.140625,2,0.454545,0.013744,西,0.0,2,...,西,5,0.400000,0.200000,0.400000,0.006872,1.375000,221,1,2
98,99,4,1021,0.269531,0,0.581818,0.013860,东南,0.0,3,...,东南,7,0.428571,0.285714,0.285714,0.004620,0.000000,322,1,2
